In [1]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram

import numpy as np
from scipy.optimize import minimize

In [2]:
# dij需求i到设施j的成本
d = [[1, 2], [1, 2]]
n = 2   # 两个设施点
m = 2   # 两个需求点
# d = [[1, 2], [3, 4], [5, 6]]
# n = 2   # 两个设施点
# m = 3   # 三个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]


In [3]:
penalty = 40
depth = 5
params = np.ones(depth * 2)

In [4]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

In [5]:
# dtype=np.complex128
def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H


def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    # print(num_qubits)
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j - 1] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2

    # for i in range(m):
    #     Ht = np.zeros((2**num_qubits, 2**num_qubits))
    #     for j in range(n):
    #         Ht += (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2
    #     Ht -= np.eye(2**num_qubits)
    #     Hp += -penalty * Ht @ Ht
    
    for i in range(m):
        for j in range(n):
            Ht = (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + m + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, j)) / 2
            Hp += -penalty * Ht @ Ht

    return Hp

In [6]:
def gnrt_Hd(n):
  Hd = np.zeros((2**n, 2**n)).astype(np.complex128)
  for i in range(n):
    j = (i + 1) % n
    Hd += (add_in_target(n, i, GateX) @ add_in_target(n, j, GateX) + add_in_target(n, i, GateY) @ add_in_target(n, j, GateY))
  return -Hd


In [7]:
from scipy.linalg import expm
def build_circ(n, m, d, g, params):
  qc = QuantumCircuit(num_qubits)
  beta = params[:depth]
  gamma = params[depth:]
  for i in range(m * n):
    qc.h(i)
  for i in range(m * n, m * n * 2, 2):
    qc.x(i)
  for i in range(m * n * 2, num_qubits):
    qc.h(i)
  for dp in range(depth):
    qc.unitary(expm(-1j * gamma[dp] * generate_Hp(n, m, d, g)), range(num_qubits)) # transpile
    # 
    for i in range(m * n):
      qc.rx(beta[dp], i)
    for i in range(m):
      qc.unitary(expm(-1j * beta[dp] * gnrt_Hd(n)), range(n * (m + i) , n * (m + i + 1)))
    for i in range(2 * m * n, num_qubits):
      qc.rx(beta[dp], i)
  qc.measure_all()
  return qc

In [8]:
print(build_circ(n, m, d, g, np.full(depth * 2, np.pi/4)))

         ┌───┐┌──────────┐┌─────────┐ ┌──────────┐┌─────────┐ ┌──────────┐»
    q_0: ┤ H ├┤0         ├┤ Rx(π/4) ├─┤0         ├┤ Rx(π/4) ├─┤0         ├»
         ├───┤│          │├─────────┤ │          │├─────────┤ │          │»
    q_1: ┤ H ├┤1         ├┤ Rx(π/4) ├─┤1         ├┤ Rx(π/4) ├─┤1         ├»
         ├───┤│          │├─────────┤ │          │├─────────┤ │          │»
    q_2: ┤ H ├┤2         ├┤ Rx(π/4) ├─┤2         ├┤ Rx(π/4) ├─┤2         ├»
         ├───┤│          │├─────────┤ │          │├─────────┤ │          │»
    q_3: ┤ H ├┤3         ├┤ Rx(π/4) ├─┤3         ├┤ Rx(π/4) ├─┤3         ├»
         ├───┤│          │├─────────┴┐│          │├─────────┴┐│          │»
    q_4: ┤ X ├┤4         ├┤0         ├┤4         ├┤0         ├┤4         ├»
         └───┘│  Unitary ││  Unitary ││  Unitary ││  Unitary ││  Unitary │»
    q_5: ─────┤5         ├┤1         ├┤5         ├┤1         ├┤5         ├»
         ┌───┐│          │├──────────┤│          │├──────────┤│          │»
    q_6: ┤ X

In [9]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(m):
    for j in range(n):
      C += d[i][j] * num[n * (1 + i) + j]
      
  for j in range(n):
    C += g[j] * num[j]

  # for i in range(m):
  #   t = 0
  #   for j in range(n):
  #     t += num[n * (1 + i) + j]
  #   C += penalty * (t - 1)**2

  for i in range(m):
    for j in range(n):
      C += penalty * (num[n * (1 + i) + j] + num[n * (1 + m + i) + j] - num[j]) ** 2
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(n, m, d, g, theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [10]:
cost_function('1010100000')

4

In [11]:
from numpy.lib.utils import source
from scipy.optimize import minimize
import numpy as np
# 初始化迭代计数器
iteration_count = 0
def test(pen, dep, par):
  global penalty, depth, params, iteration_count
  iteration_count = 0
  penalty = pen
  depth = dep
  params = par
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      if iteration_count % 10 == 0:
          print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations = 1000

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, params, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')
  backend.shots = 100000

  shots=100000
  qc_res = build_circ(n, m, d, g, params=res.x)

  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  print("\n----------------- Full result ---------------------")
  print("selection\t\tprobability\tvalue")
  print("---------------------------------------------------")
  for x in sorted_counts[:20]:
    print(x, "{:.1f}%".format(counts[x] / shots * 100), cost_function(x))

In [12]:
test(40, 5, np.full(5 * 2, np.pi/5))

Iteration 10, Result: 115.865
Iteration 20, Result: 164.5915
Iteration 30, Result: 91.3415
Iteration 40, Result: 149.9485
Iteration 50, Result: 123.7325
Iteration 60, Result: 79.2765
Iteration 70, Result: 84.0025
Iteration 80, Result: 81.2145
Iteration 90, Result: 80.1465
Iteration 100, Result: 80.1145
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 80.1145
       x: [ 5.784e-01  8.813e-01  1.715e+00  5.378e-01  4.906e-01
            6.250e-01  6.197e-01  1.646e+00  6.151e-01  1.641e+00]
    nfev: 100
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
1010100000 24.8% 4
1110100000 8.3% 85
1010101000 5.9% 44
1010100010 5.8% 44
0010100000 4.4% 82
1010010001 2.0% 205
1001100100 2.0% 205
1110010001 1.6% 126
1101100100 1.5% 126
1010101010 1.5% 84
0110100000 1.4% 163
0010010000 1.2% 83
0001100000 1.1% 83
1010101100 1.0% 84
1010100011 1.0% 84
0010

In [13]:
test(40, 3, np.full(3 * 2, np.pi/3))

Iteration 10, Result: 157.825
Iteration 20, Result: 160.784
Iteration 30, Result: 182.521
Iteration 40, Result: 104.64
Iteration 50, Result: 103.0725
Iteration 60, Result: 102.61
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 102.751
       x: [ 1.228e+00  2.063e+00  1.102e+00  1.036e+00  8.981e-01
            1.046e+00]
    nfev: 65
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
1010100000 6.1% 4
1110100000 4.5% 85
1010100100 4.4% 44
1010100001 4.3% 44
1110100101 4.1% 5
1010100101 3.4% 84
1110100001 3.0% 45
1110100100 3.0% 45
1010101000 1.9% 44
1010100010 1.9% 44
0101100001 1.6% 44
0110010100 1.6% 44
1110101001 1.5% 85
1110100110 1.5% 85
1101100001 1.3% 46
1110010100 1.2% 46
1110101101 1.2% 45
1110100111 1.2% 45
0110100000 1.1% 163
0010100010 1.0% 202


In [14]:
# for i in range(1, 20):
#   print(f'depth == {i}')
#   test(20, i, np.ones(2 * i))